# PUMS Household Income vs. AMI (2021) in Phoenix  PUMAS for 2 bedroom

- https://www.census.gov/data/developers/data-sets/

For households by income and household size to compare to HUD AMI in same year
-  https://api.census.gov/data/2021/acs/acs1/pums/variables.html

2021 AMI by HH Size (from City of Phoenix)
- https://www.phoenix.gov/humanservicessite/Documents/2021%20AMI%20Limits%204.2.21.pdf

In [1]:
import pandas as pd
import math
import numpy as np
import os

In [2]:
import get_pums as get
import pums as calc
from AMI_BANDS_2021 import *

In [3]:
#Search parameters
y1 = '2021'

sample = 'acs1'

phx_pumas = ['0400113','0400114','0400115','0400116','0400117',
             '0400118','0400119','0400120','0400121','0400122','0400123',
             '0400125','0400128','0400112','0400129']

#variables needed: household size NP, income INCP, & income adjustment factor
data_cols = 'SERIALNO,ST,PUMA,HINCP,NP,WGTP,ADJINC'

In [4]:
#assuming July 2021 $1 for HHInc equivalent to estimate 2023 Inc equivalent
inflation_adjust = 1.11

In [5]:
#if there's time, it would be better to upload a dict of prices from MAG
#then generate the dictionary through calcs in python (vs. excel)
#flag for later

#Income to afford median household rent price + utilities by PUMA
#3 bedroom multifamily unit
inc_needed_mf = {'0400112':108100,'0400113':82800,'0400114':68770 ,'0400115':64285,
              '0400116':63250,'0400117':75900,'0400118':76475,'0400119':56925,
              '0400120':74750,'0400121':58650,'0400122':54970,'0400123':54970,
              '0400125':55154,'0400128':66700,'0400129':79350}

 
#Income to afford median household sales price by PUMA
#3 bedroom single-family unit
inc_needed_sf = {'0400112':110308,'0400113':83214,'0400114':67390,'0400115':63894,
              '0400116':56856,'0400117':65918,'0400118':70334,'0400119':50876,
              '0400120':79350,'0400121':62238,'0400122':52164,'0400123':68540,
              '0400125':55706,'0400128':72634,'0400129':80960}

In [6]:
# create a list of replicate weights
repwt = 'WGTP'
repwts = [repwt+str(i) for i in range(1, 81)]

## Get PUMA data

In [7]:
df = get.get_puma(sample,y1,data_cols)

KeyboardInterrupt: 

In [ ]:
df['GEO_ID'] = df['ST']+df['PUMA']
df = df[df.GEO_ID.isin(phx_pumas)]
df  = df.drop(['SERIALNO','ST','PUMA'],axis=1)
df = df[['GEO_ID']+[col for col in df.columns if col != 'GEO_ID']] #move id to first col
for col in df.columns[1:]: df[col] = df[col].astype(float)

In [ ]:
df['HHSz'] = pd.cut(df['NP'],bins=[0,1,2,3,4,5,6,7,14],
                   labels=['1','2','3','4','5','6','7','8'])
df['HHSz'] = df['HHSz'].astype(str)

In [ ]:
df['inc_needed_sf'] = df['GEO_ID'].map(inc_needed_sf)
df['inc_needed_mf'] = df['GEO_ID'].map(inc_needed_mf)

In [ ]:
dff = df[~(df.HHSz.isna())&(df.HINCP!=-60000)&(df.HINCP!=0)].copy()
dff['HINCP'] = dff.ADJINC * dff.HINCP * inflation_adjust

In [ ]:
dff['can_rent_sf'] = np.where(dff.HINCP>=dff.inc_needed_sf,'can rent','cannot afford')
dff['can_rent_mf'] = np.where(dff.HINCP>=dff.inc_needed_mf,'can rent','cannot afford')

In [ ]:
#assign AMI range based on household size and ami dictionaries
dff['AMI_range'] = np.where((dff['HINCP']<=dff['HHSz'].map(AMI_30pct)),inc_lbl[0],
                   np.where((dff['HINCP']>dff['HHSz'].map(AMI_30pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_50pct)),inc_lbl[1],
                   np.where((dff['HINCP']>dff['HHSz'].map(AMI_50pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_80pct)),
                   inc_lbl[2],np.where((dff['HINCP']>dff['HHSz'].map(AMI_80pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_100pct)),
                   inc_lbl[3],np.where((dff['HINCP']>dff['HHSz'].map(AMI_100pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_120pct)),
                                       inc_lbl[4],inc_lbl[5])))))

In [ ]:
#make a column for each PUMA that assesses the number of households 
sf_cols=[]
for k in inc_needed_sf.keys():
    dff[f'aff_{k}_can_sf']=np.where(dff.HINCP>=inc_needed_sf[k],1,0)
    dff[f'aff_{k}_cant_sf']=np.where(dff.HINCP<inc_needed_sf[k],1,0)
    dff[f'aff_{k}_can_sf_wt']=dff[f'aff_{k}_can_sf']*dff.WGTP
    dff[f'aff_{k}_cant_sf_wt']=dff[f'aff_{k}_cant_sf']*dff.WGTP
    sf_cols = sf_cols + [f'aff_{k}_can_sf',f'aff_{k}_cant_sf',
                        f'aff_{k}_can_sf_wt',f'aff_{k}_cant_sf_wt']
    #sf_cols.append(f'aff_{k}_can_sf')
    #sf_cols.append(f'aff_{k}_cant_sf')
    #sf_cols.append(f'aff_{k}_can_sf_wt')
    #sf_cols.append(f'aff_{k}_cant_sf_wt')

In [ ]:
#make a column for each PUMA that assesses the number of households 
mf_cols=[]
for k in inc_needed_mf.keys():
    dff[f'aff_{k}_can_mf']=np.where(dff.HINCP>=inc_needed_mf[k],1,0)
    dff[f'aff_{k}_cant_mf']=np.where(dff.HINCP<inc_needed_mf[k],1,0)
    dff[f'aff_{k}_can_mf_wt']=dff[f'aff_{k}_can_mf']*dff.WGTP
    dff[f'aff_{k}_cant_mf_wt']=dff[f'aff_{k}_cant_mf']*dff.WGTP
    mf_cols = mf_cols+[f'aff_{k}_can_mf',f'aff_{k}_cant_mf',
            f'aff_{k}_can_mf_wt',f'aff_{k}_cant_mf_wt']

In [ ]:
dff.head(3)

### table by PUMA for renters by AMI range - can afford/can't afford median rent

In [ ]:
def make_est(df):
    df['hh_SE'] = df.apply(lambda x: (calc.get_se(x['WGTP'],x[repwts])),axis=1)
    df['hh_MOE'] = df.apply(lambda x: (calc.get_moe(x['hh_SE'])),axis=1)
    df['hh_CV'] = df.apply(lambda x: (calc.get_cv(x['WGTP'],x['hh_SE'])),axis=1)
    df.rename(columns={'WGTP':'hh'},inplace=True)
    return df

In [ ]:
drop_cols = ['HINCP','NP','ADJINC','HHSz','inc_needed_sf','inc_needed_mf']

In [ ]:
table = dff.copy().drop(columns=drop_cols)

In [ ]:
rent_ami = table.copy().drop(columns=sf_cols+mf_cols).groupby(['GEO_ID','AMI_range','can_rent_sf','can_rent_mf']).sum().reset_index()
rent_ami = make_est(rent_ami)
rent_ami = rent_ami.drop(columns=repwts)
rent_ami = pd.pivot_table(rent_ami,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',\
                          columns=['AMI_range','can_rent_sf','can_rent_mf'],aggfunc=np.sum).reset_index()

In [ ]:
rent_tot = table.copy().drop(columns=sf_cols+mf_cols+['AMI_range']).groupby(['GEO_ID','can_rent_sf','can_rent_mf']).sum().reset_index()
rent_tot = make_est(rent_tot)
rent_tot = rent_tot.drop(columns=repwts)
rent_tot = pd.pivot_table(rent_tot,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',
                          columns=['can_rent_sf','can_rent_mf'],aggfunc=np.sum).reset_index()

In [ ]:
table_sf = table.copy().drop(columns=['AMI_range','can_rent_sf','can_rent_mf']+mf_cols).groupby(['GEO_ID']).sum().reset_index()
table_sf = make_est(table_sf)
table_sf = table_sf.drop(columns=repwts)

In [ ]:
table_mf = table.copy().drop(columns=['AMI_range','can_rent_sf','can_rent_mf']+sf_cols).groupby(['GEO_ID']).sum().reset_index()
table_mf = make_est(table_mf)
table_mf = table_mf.drop(columns=repwts)

In [ ]:
table_sf.head()

In [ ]:
with pd.ExcelWriter(f'../../output/pums_gap_rent/pums_rent_gap_marketdata_120AMi_inflation.xlsx') as writer:
    rent_ami.to_excel(writer,sheet_name='can_cannot_withinpuma_ami.xlsx')
    rent_tot.to_excel(writer,sheet_name='can_cannot_withinpuma.xlsx')
    table_sf.to_excel(writer,sheet_name='can_cannot_all_sf.xlsx')
    table_mf.to_excel(writer,sheet_name='can_cannot_all_mf.xlsx')